# Load an artist portrait model

In [2]:
import numpy as np

artist_path = "AF_dataset/Amedeo_Modigliani/"
mu_artist = np.load(artist_path + "mean.npy")
cov_artist = np.load(artist_path + "cov.npy")

In [3]:
# sample an offset vector from the distribution
v_artist = np.random.multivariate_normal(mu_artist, cov_artist)
x_offset = v_artist[0:68]
y_offset = v_artist[68:]

# Load Natural Image and Landmarks

In [4]:
natural_img = "300w/indoor_007.png"
natural_pts = "300w/indoor_007.pts"

import os
import cv2
import numpy as np

with open(natural_pts) as pts_file:
    data = pts_file.readlines()[3:68+3]
    data = [np.array(line.split(), dtype=float) for line in data]
    landmarks = np.array(data) # x is cols y is rows
    
print(landmarks.shape)
image = cv2.imread(natural_img)
im_rows = image.shape[0]
im_cols = image.shape[1]

(68, 2)


# Match and Perturb Landmarks

In [5]:
# normalize and center landmarks
x_landmarks = landmarks[:,0]
y_landmarks = landmarks[:,1]

x_landmarks /= im_cols
y_landmarks /= im_rows

x_landmarks -= x_landmarks[33]
y_landmarks -= y_landmarks[33]

In [6]:
# add offset
l_x = x_landmarks + x_offset
l_y = y_landmarks + y_offset

In [7]:
# map the landmarks back into the original image
l_x += x_landmarks[33]
l_y += y_landmarks[33]
l_x *= im_cols
l_y *= im_rows

# Apply Style Transfer With TPS Interpolation

In [27]:
control_points = np.zeros((68,2))
target_points = np.zeros((68,2))
control_points[:,0] = x_landmarks
control_points[:,1] = y_landmarks
target_points[:,0] = x_landmarks
target_points[:,1] = y_landmarks
    
matches = list()
for ipoint in range(0, control_points.shape[1]):
    matches.append(cv2.DMatch(ipoint, ipoint, 0))
    
    
control_points = np.array([control_points])
target_points = np.array([target_points])
    
tps = cv2.createThinPlateSplineShapeTransformer()
tps.estimateTransformation(control_points, target_points, matches)
warped = tps.warpImage(image)

In [28]:
cv2.imshow("test", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("warped", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()